In [24]:
import sys
from pathlib import Path
import numpy as np
import scipy

repo_root = None
for path in [Path.cwd().resolve()] + list(Path.cwd().resolve().parents):
    if (path / "cp_reach").exists():
        repo_root = path
        break
if repo_root is None:
    raise RuntimeError("Could not locate repo root")
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Updated imports for new structure
from cp_reach.reachability import (
    sympy_load,
)
from cp_reach.dynamics import classification
import sympy as sp

### Beginning of Code for EKF Comparison. Once the symbolic jacobian was obtained, I continued trying to fix the IEKF. Note the following code requires the latest branch of CP_Reach: https://github.com/micahkc/cp_reach and the IR branch of Cyecca: https://github.com/CogniPilot/cyecca

In [2]:
%load_ext autoreload
%autoreload 2

### Import model dynamics and states from Modelica

In [22]:
# Load SymPy backend (include outputs so error dynamics can be computed)
model_sympy = sympy_load("modelica_models/quadrotor2.mo", output_names=[])
states = model_sympy.states
inputs = model_sympy.inputs

ss = model_sympy.symbolic
f = ss.f + ss.Bu # This is the function
sympy_backend = model_sympy.backend
state_symbols = [sympy_backend.symbols[var.name] for var in sympy_backend.model.states]
input_symbols = [sympy_backend.symbols[var.name] for var in sympy_backend.model.inputs]
param_symbols = [sympy_backend.symbols[var.name] for var in sympy_backend.model.parameters]
print(f'states: {state_symbols}')
print(f'inputs: {inputs}')
f

states: [x, y, h, U, V, W, phi, theta, psi]
inputs: ['P', 'Q', 'R']


Matrix([
[U*cos(psi)*cos(theta) + V*sin(phi)*sin(theta)*cos(psi) - V*sin(psi)*cos(phi) + W*sin(phi)*sin(psi) + W*sin(theta)*cos(phi)*cos(psi)],
[U*sin(psi)*cos(theta) + V*sin(phi)*sin(psi)*sin(theta) + V*cos(phi)*cos(psi) - W*sin(phi)*cos(psi) + W*sin(psi)*sin(theta)*cos(phi)],
[                                                                       U*sin(theta) - V*sin(phi)*cos(theta) - W*cos(phi)*cos(theta)],
[                                                                                                          -Q*W + R*V - g*sin(theta)],
[                                                                                                  P*W - R*U + g*sin(phi)*cos(theta)],
[                                                                                                 -P*V + Q*U + g*cos(phi)*cos(theta)],
[                                                                                  P + Q*sin(phi)*tan(theta) + R*cos(phi)*tan(theta)],
[                                             

In [ ]:
# print(sp.latex(f)) # Latex export

\left[\begin{matrix}U \cos{\left(\psi \right)} \cos{\left(\theta \right)} + V \sin{\left(\phi \right)} \sin{\left(\theta \right)} \cos{\left(\psi \right)} - V \sin{\left(\psi \right)} \cos{\left(\phi \right)} + W \sin{\left(\phi \right)} \sin{\left(\psi \right)} + W \sin{\left(\theta \right)} \cos{\left(\phi \right)} \cos{\left(\psi \right)}\\U \sin{\left(\psi \right)} \cos{\left(\theta \right)} + V \sin{\left(\phi \right)} \sin{\left(\psi \right)} \sin{\left(\theta \right)} + V \cos{\left(\phi \right)} \cos{\left(\psi \right)} - W \sin{\left(\phi \right)} \cos{\left(\psi \right)} + W \sin{\left(\psi \right)} \sin{\left(\theta \right)} \cos{\left(\phi \right)}\\U \sin{\left(\theta \right)} - V \sin{\left(\phi \right)} \cos{\left(\theta \right)} - W \cos{\left(\phi \right)} \cos{\left(\theta \right)}\\- Q W + R V - g \sin{\left(\theta \right)}\\P W - R U + g \sin{\left(\phi \right)} \cos{\left(\theta \right)}\\- P V + Q U + g \cos{\left(\phi \right)} \cos{\left(\theta \right)}\\P + Q \s

### Solve Jacobian

In [5]:
jac = f.jacobian(state_symbols)
jac

Matrix([
[0, 0, 0, cos(psi)*cos(theta), sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi),  sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi),  V*sin(phi)*sin(psi) + V*sin(theta)*cos(phi)*cos(psi) - W*sin(phi)*sin(theta)*cos(psi) + W*sin(psi)*cos(phi), -U*sin(theta)*cos(psi) + V*sin(phi)*cos(psi)*cos(theta) + W*cos(phi)*cos(psi)*cos(theta), -U*sin(psi)*cos(theta) - V*sin(phi)*sin(psi)*sin(theta) - V*cos(phi)*cos(psi) + W*sin(phi)*cos(psi) - W*sin(psi)*sin(theta)*cos(phi)],
[0, 0, 0, sin(psi)*cos(theta), sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi), -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi), -V*sin(phi)*cos(psi) + V*sin(psi)*sin(theta)*cos(phi) - W*sin(phi)*sin(psi)*sin(theta) - W*cos(phi)*cos(psi), -U*sin(psi)*sin(theta) + V*sin(phi)*sin(psi)*cos(theta) + W*sin(psi)*cos(phi)*cos(theta),  U*cos(psi)*cos(theta) + V*sin(phi)*sin(theta)*cos(psi) - V*sin(psi)*cos(phi) + W*sin(phi)*sin(psi) + W*sin(theta)*cos(phi)*cos(psi)],
[0, 0, 0,          sin(theta),                           

In [ ]:
# Initial conditions
x0 = [0,0,0,0,0,0,0,0,0]

# Process noise on velocity only
dt_noise = 0.01
tf = 60

# Process noise
process_sigma = 0.01
Q = np.zeros((9,9))
Q[3:6,3:6] = process_sigma**2*np.eye(3)
rng = np.random.default_rng(seed=0)

# Zero order hold - process noise is constant over delta t.
process_noise = rng.multivariate_normal(np.zeros(9), Q, size=int(tf/dt_noise))

G = sp.eye(9)
w1, w2, w3, w4, w5, w6, w7, w8, w9 = sp.symbols('w1 w2 w3 w4, w5, w6, w7, w8, w9')
v = sp.Matrix([w1, w2, w3, w4, w5, w6, w7, w8, w9 ])

omega1,omega2,omega3 = sp.symbols("P, Q, V")
omega = sp.Matrix([omega1, omega2, omega3])

f_v = f + G@v
lam_f = sp.lambdify([state_symbols, omega, v], f_v[:])

def dynamics(t,x):
    noise = process_noise[int(t/dt_noise)]
    res = lam_f(x,noise)
    # print(res)
    return res

### Integrate for 60s

In [23]:
# dt=0.1
# time = np.arange(0,tf, dt)
# t_span = (time[0], time[-1])
# # sol = solve_ivp(lam_f, t_span, x0, t_eval=np.linspace(0, 15, 300))
# res = scipy.integrate.solve_ivp(
#                 fun=lambda t,y: dynamics(t,y),
#                 t_span=t_span,
#                 y0=x0,
#                 t_eval=time,
#             )